In [2]:
import sys
sys.path.append('../')
import requests
import ipywidgets as widgets
import pandas as pd
import json
from ipywidgets import Layout, AppLayout, Button, ToggleButtons
from pprint import pprint
import cfg.sparql as sparql

In [10]:
from IPython.display import clear_output, display
import time
import ipywidgets as widgets
from functools import lru_cache

# set up a dictionary of Output widgets
outputs = {i: widgets.Output() for i in range(0,3)}

# add the Output widgets as tab childen
tab = widgets.Tab()
tab.children = list(outputs.values())
for i, title in outputs.items():
    tab.set_title(i, 'Tab '+str(i))

def print_on_select(widget):
# get the correct Output widget based on the index of the chosen tab
    tab_idx = widget['new']
    output_widget = outputs[tab_idx]
    with output_widget:
        clear_output()
        print('running long function')
        value = long_running_function(tab_idx)
        clear_output()
        print(value)

@lru_cache(32)
def long_running_function(tab_idx):
        time.sleep(2)
        return 'this is tab number ' + str(tab_idx)

tab.observe(print_on_select, names='selected_index')

display(tab)

In [6]:
query = sparql.QUERY_ASSESSMENTS

print(query)


        PREFIX camssa: <http://data.europa.eu/2sa/assessments/>
        PREFIX cssvrsc: <http://data.europa.eu/2sa/cssv/rsc/>
        PREFIX sc: <http://data.europa.eu/2sa/scenarios#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX cav: <http://data.europa.eu/2sa/cav#>
        PREFIX camss: <http://data.europa.eu/2sa#>
        PREFIX cssv: <http://data.europa.eu/2sa/cssv#>
        PREFIX dct: <http://purl.org/dc/terms/>
        
        select distinct ?Assessment ?Scenario
        where {
            ?assObj rdf:type cav:Assessment ;
                cav:contextualisedBy ?scObj ;
                dct:title ?Assessment .
            ?scObj rdf:type cav:Scenario ;
                dct:title ?Scenario.
        }
    


In [5]:
# GLOBAL VARIABLES

URL_MICROSERVICE= 'http://localhost:5100/assessments/get_assessments'

# UI COMPONENTS

# Toggle buttons for default queries selection
button_option= ToggleButtons(options=['Query A', 'Query B', 'Query C', 'Query D'])
display(button_option)

def changed(change):
    button_value = button_option.value
    query_1 = 'Query a'
    query_2 = 'Query b'
    query_3 = 'Query c'
    query_4 = 'Query d'
    
    if button_value == 'Query A':
        TEXT_AREA.value = query_1
        print(query_1)
    elif button_value == 'Query B':
        TEXT_AREA.value = query_2
        print(query_2)
    elif button_value == 'Query C':
        TEXT_AREA.value = query_3
        print(query_3)
    elif button_value == 'Query D':
        TEXT_AREA.value = query_4
        print(query_4)
        
button_option.observe(changed, 'value')

# Text area to hold query value
TEXT_AREA = widgets.Textarea(value='', placeholder='Type a query', layout=Layout(width='100%',height='300px'))
display(TEXT_AREA)

# Button to send query

button= widgets.Button(description='Run')
output= widgets.Output()
display(button, output)

# QUERY FUNCTIONS

# Method to execute query
@output.capture()
def send_query(q):
    query= TEXT_AREA.value
    params= {'sparql_query': query}
    msg= requests.post(url= URL_MICROSERVICE, params= params)
    response_json = msg.json()
    df_assessments = pd.DataFrame(response_json, columns=['Assessments', 'Scenario'])
    pprint(df_assessments)
    
    
button.on_click(send_query)

ToggleButtons(options=('Query A', 'Query B', 'Query C', 'Query D'), value='Query A')

Textarea(value='', layout=Layout(height='300px', width='100%'), placeholder='Type a query')

Button(description='Run', style=ButtonStyle())

Output()